<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT21/blob/Diracturing/Lab_3/Diracturing_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3**
**Julius Andersson**

# **Abstract**
In this lab 3 algorithms were constructed and tested. They were the Jacobi iteration, Gauss-Seidel iteration and lastly Newton's method. All algorithms did pass the tests.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [ ]:
# Copyright (C) 2020 Julius Andersson

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [ ]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import math

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**
The theme for this lab was iterated methods. Previous we dealt with algorithms which produce exact solutions except for rounding errors. Now instead we use algorithms which should get closer to the solution for each iteration. The Jacobi iteration & Gauss-Seidel iteration algorithms are used for solving system of linear equations while the Newton algorithm are used to solve scalar nonlinear equation of the form f(x)=0.

# **Method**

1) Jacobi iteration 

The Jacobi iteration works similiar to the Richardson iteration but it is left preconditioned. Richardson iteration converges if $||I-\alpha A||<1$ and this term is also connected to the rate of convergence. Jacobi iteration uses left precondition so the convergence critera instead is $||I-D^{-1}A||<1$, where D has the same diagonal as A but every other element is zero. If A is diagonally dominant this can be a way of lowering the convergence rate.

The algorithm presented follows from the explanations at page 150-151 from the lecture notes. 

In [84]:
def jacobi(A,b):
  x=np.zeros(b.shape)
  Tolerance=1e-10
  
  while np.linalg.norm(A@x-b)>Tolerance:
    D=np.diagflat(np.diagonal(A))
    Dinv=np.linalg.inv(D)
    x+=-Dinv@A@x+Dinv@b
 
  return x


In [ ]:
##### nothing too see#########
A=np.array([[2,2,2],[1,2,4],[10,3,5]])
np.linalg.eigvals(A)
np.linalg.svd(A)

0.0001


2) Gauss-Seidel iteration 

Similar to Jacobi iteration but uses another left preconditioner. Here we using L which is a lower triangular matrix. L is taken to be the lower triangular part of matrix A and zero elsewhere. While the jacobi iteration method can be made to be embarrasingly parallel the Gauss-Seidel can not. Instead it can be made to use lower storage [1]. 

The algorithm presented follows from the explanations at page 152 from the lecture notes.

In [ ]:
def gase(A,b):
  x=np.zeros(b.shape)
  Tolerance=1e-10
  
  while np.linalg.norm(A@x-b)>Tolerance:
    L=np.tril(A)
    Linv=np.linalg.inv(L)
    x+=-Linv@A@x+Linv@b
  return x

3) Newton's method

Newton's method uses fixed point iteration $x^{(k+1)}=x^{(k)}+\alpha f(x^{(k)})$. Newton method uses $\alpha=-f'(x^{(k)})^{-1}$ which make it to a quadratic order of convergence (if f(x) meets certain requirements). 

The algorithm is from algorithm 8.2 in the lecture notes.  

In [ ]:
def der(f,x):
  h=1e-8
  return (f(x+h)-f(x))/h

def newton(f,x0):
  Tolerance=1e-10
  x=x0
  while abs(f(x))>Tolerance:
    df=der(f,x)
    x=x-f(x)/df
  return x



# **Results**

In [ ]:
##########Tests for jacobi and gauss-seidel ################
A=np.array([[100,2,0],[5,55,2],[3,1,78]])
y=np.array([3,5,2])
b=A@y

A2=np.array([[353,5,9],[5,152,10],[23,4,178]])
y2=np.array([10,3,25])
b2=A2@y2

x=jacobi(A,b)
x1=gase(A,b)

x2=jacobi(A2,b2)
x3=gase(A2,b2)


assert np.isclose(np.linalg.norm(x-y),0,atol=1e-10)
assert np.isclose(np.linalg.norm(x1-y),0,atol=1e-10)

assert np.isclose(np.linalg.norm(x2-y2),0,atol=1e-10)
assert np.isclose(np.linalg.norm(x3-y2),0,atol=1e-10)



########### Tests for Newton method ################
def f1(x):
  return math.cos(x)

def f2(x):
  return x-3

def f3(x):
  return x**2-1


assert np.isclose(abs(newton(f1,0.5)-np.pi/2),0,atol=1e-10)
assert np.isclose(abs(newton(f2,0.5)-3),0,atol=1e-10)
assert np.isclose(abs(newton(f3,0.5)-1),0,atol=1e-10)


print("ok")


ok


As can be seen above all methods passed the tests.

# **Discussion**
There is not much to say about the results. We could see all methods converge to the actual value. One thing one can notice is that newton's method only finds 1 solution and it might be hard to find all roots. Improvements could be to calculate if the convergence criteria is fulfilled before calling the actual algorithm or to introduce a max time each algorithm can run.   

# **APPENDIX**

# **References**
[1] https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method